In [1]:
pip install torchvision

     |████████████████████████████████| 17.4 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 804.1 MB 112.7 MB/s eta 0:00:01   |████████████▏                   | 306.7 MB 3.3 MB/s eta 0:02:33     |████████████████████▌           | 516.1 MB 74.6 MB/s eta 0:00:04███▊        | 595.1 MB 79.6 MB/s eta 0:00:03██████████████████████▊       | 622.2 MB 79.6 MB/s eta 0:00:03��█████████████▍    | 688.4 MB 75.4 MB/s eta 0:00:02██████████████████▌   | 716.8 MB 75.4 MB/s eta 0:00:02 eta 0:00:02B/s eta 0:00:01MB 105.4 MB/s eta 0:00:01██████▏ | 758.4 MB 105.4 MB/s eta 0:00:01     |██████████████████████████████▋ | 767.9 MB 105.4 MB/s eta 0:00:01██████▊ | 771.4 MB 105.4 MB/s eta 0:00:01█▎| 786.8 MB 112.7 MB/s eta 0:00:01��██████████████████████████▉| 801.2 MB 112.7 MB/s eta 0:00:01Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install markdown

     |████████████████████████████████| 97 kB 2.9 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

     |████████████████████████████████| 394.3 MB 23 kB/s s eta 0:00:01     |██████████████████▏             | 224.0 MB 40.2 MB/s eta 0:00:05
     |████████████████████████████████| 2.9 MB 65.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 52.3 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 51.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 52.0 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 41.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 142 kB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 47.0 MB/s eta 0:00:01
     |███████████████████████████

In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install --user --upgrade tensorflow-gpu

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
import tensorflow as tf
import random
import numpy as np
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def getZerosAndOnes(fileName, electrod, trainslabel):
    data = pd.read_csv(fileName)
    session= data[[electrod,'FeedBackEvent']]
    labels = pd.read_csv(trainslabel) 
    eventsIndexes=[]
    for index, row in session.iterrows():
        if row['FeedBackEvent']==1:
            eventsIndexes.append(index)
    eventsValues=[]
    for index, row in labels.iterrows():
        if row['IdFeedBack'].startswith('S02_Sess01'):
            eventsValues.append(row['Prediction'])   
    dataEpoches=[]
    for (index, val) in enumerate(eventsIndexes):
        if eventsValues[index]==1:
            dataEpoches.append(session[val-120:val+80])
    moreDataEpoches=[]
    for (index, val) in enumerate(eventsIndexes):
        if index>0 and index<len(eventsIndexes)-1:
            count=1
            while val+80+count*200<eventsIndexes[index+1]-120:
                count+=1
                moreDataEpoches.append(session[val+80+count*200:val+80+count*200+200])
    Xm=[]
    for row in moreDataEpoches:
        Xm.append(row[electrod])
    Xm=np.array(Xm)
    Xs=[]
    for row in dataEpoches:
        Xs.append(row[electrod])
    Xs=np.array(Xs)
    y_dataZero=np.zeros(len(Xm))
    y_dataOne=np.ones(len(Xs))
    x_dataZero=Xm
    x_dataOne=Xs
    return x_dataOne, y_dataOne, x_dataZero, y_dataZero

In [3]:
x_dataOne, y_dataOne, x_dataZero, y_dataZero=getZerosAndOnes("data.csv", 'Fp2', "TrainLabels.csv")


In [4]:
print(x_dataOne.shape)
print(y_dataOne.shape)
print(x_dataZero.shape)
print(y_dataZero.shape)

(50, 200)
(50,)
(491, 200)
(491,)


In [137]:
def load_data_from_arrays(one, zero, y_one, y_zero, train_test_split=0.9):
    data_size_one = len(one)
    data_size_zero = len(zero)
    data_size=data_size_one+data_size_zero
    test_size = int(data_size - round(data_size_zero * train_test_split)- round(data_size_one * train_test_split))
    print("Data size: {}".format(data_size))
    print("\nTest size: {}".format(test_size))
    
    data_one=one[0:round(data_size_one * train_test_split)]
    data_zero=zero[0:round(data_size_zero * train_test_split)]
    data_train=np.concatenate((data_one, data_zero), axis=0)
    train=pd.DataFrame(data_train)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\nTraining set:")
    x_train = train
    
    data_one=y_one[0:round(data_size_one * train_test_split)]
    data_zero=y_zero[0:round(data_size_zero * train_test_split)]
    data_train=np.concatenate((data_one, data_zero), axis=0)
    train=pd.DataFrame(data_train)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\t - x_train: {}".format(len(x_train)))
    y_train = train
    print("\t - y_train: {}".format(len(y_train)))
    
    
    data_one=one[round(data_size_one * train_test_split):data_size_one]
    data_zero=zero[round(data_size_zero * train_test_split):data_size_zero]
    data_test=np.concatenate((data_one, data_zero), axis=0)
#     random.shuffle(data_test)
    test=pd.DataFrame(data_test)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\nTesting set:")
    x_test = test
    
    data_one=y_one[round(data_size_one * train_test_split):data_size_one]
    data_zero=y_zero[round(data_size_zero * train_test_split):data_size_zero]
    data_test=np.concatenate((data_one, data_zero), axis=0)
#     random.shuffle(data_test)
    test=pd.DataFrame(data_test)
    
    print(data_one.shape)
    print(data_zero.shape)
    print(train.shape)
    print("\t - x_test: {}".format(len(x_test)))
    y_test = test
    print("\t - y_test: {}".format(len(y_test)))
    return x_train, y_train, x_test, y_test

In [138]:
x_train, y_train, x_test, y_test=load_data_from_arrays(x_dataOne, x_dataZero,y_dataOne,y_dataZero,0.75)

Data size: 541

Test size: 135
(38, 200)
(368, 200)
(406, 200)

Training set:
(38,)
(368,)
(406, 1)
	 - x_train: 406
	 - y_train: 406
(12, 200)
(123, 200)
(406, 1)

Testing set:
(12,)
(123,)
(406, 1)
	 - x_test: 135
	 - y_test: 135


In [139]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(406, 200)
(406, 1)
(135, 200)
(135, 1)


In [140]:
inputs = keras.Input(shape=(200,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
y = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(1, activation='sigmoid', name='output')(y)

model = keras.Model(inputs=inputs, outputs=outputs)

In [142]:
# x_train = (np.array(x_train)).reshape(406, 200).astype('float32')
# x_test = (np.array(x_test)).reshape(135, 200).astype('float32')
# y_train = (np.array(y_train)).reshape(406, 1).astype('float32')
# y_test = (np.array(y_test)).reshape(135, 1).astype('float32')
y_test

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
130,0.0
131,0.0
132,0.0
133,0.0


In [128]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['sparse_categorical_accuracy'])

print('# Обучаем модель на тестовых данных')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    # Мы передаем валидационные данные для
                    # мониторинга потерь и метрик на этих данных
                    # в конце каждой эпохи
                    validation_data=(x_test, y_test))

# Обучаем модель на тестовых данных
Epoch 1/5
7/7 [==============================] - 1s 34ms/step - loss: 17.1672 - sparse_categorical_accuracy: 0.9125 - val_loss: 8.4486 - val_sparse_categorical_accuracy: 0.9111
Epoch 2/5
7/7 [==============================] - 0s 11ms/step - loss: 7.0569 - sparse_categorical_accuracy: 0.9201 - val_loss: 6.1698 - val_sparse_categorical_accuracy: 0.9111
Epoch 3/5
7/7 [==============================] - 0s 11ms/step - loss: 7.3639 - sparse_categorical_accuracy: 0.8904 - val_loss: 2.0970 - val_sparse_categorical_accuracy: 0.9111
Epoch 4/5
7/7 [==============================] - 0s 6ms/step - loss: 4.8606 - sparse_categorical_accuracy: 0.9126 - val_loss: 6.4138 - val_sparse_categorical_accuracy: 0.9111
Epoch 5/5
7/7 [==============================] - 0s 13ms/step - loss: 3.0513 - sparse_categorical_accuracy: 0.9173 - val_loss: 9.4918 - val_sparse_categorical_accuracy: 0.9111


In [129]:

results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)
predictions = model.predict(x_test)
print('размерность прогнозов:', predictions.shape)

2/2 [==============================] - 0s 2ms/step - loss: 9.4918 - sparse_categorical_accuracy: 0.9111
test loss, test acc: [9.491764068603516, 0.9111111164093018]
размерность прогнозов: (135, 1)


In [130]:
predictions=np.round(predictions,15)

In [135]:
for (index, val) in enumerate(y_test):
    print(val," : ",predictions[index])

0  :  [0.]


In [121]:
for (index, val) in enumerate(y_test[10:]):
    print(val," : ",predictions[index])

[1.]  :  [0.]
[1.]  :  [5.5e-14]
[0.]  :  [2.0379e-11]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [4.1e-14]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [5.e-15]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [5.380454e-09]
[0.]  :  [1.961e-12]
[0.]  :  [3.804e-12]
[0.]  :  [1.e-15]
[0.]  :  [5.149e-12]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [4.3644e-11]
[0.]  :  [6.22334e-10]
[0.]  :  [1.194e-12]
[0.]  :  [4.432e-12]
[0.]  :  [1.13391005e-10]
[0.]  :  [2.73e-11]
[0.]  :  [6.87687e-10]
[0.]  :  [7.353e-12]
[0.]  :  [3.34e-13]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]  :  [0.]
[0.]